In [1]:
from llama_index.legacy import SimpleDirectoryReader, StorageContext, ServiceContext
from llama_index.legacy.indices.vector_store import VectorStoreIndex
from llama_iris import IRISVectorStore

[nltk_data] Downloading package stopwords to /Users/aryanput/dp/hackat
[nltk_data]     hon-2024/venv/lib/python3.12/site-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/aryanput/dp/hackathon-
[nltk_data]     2024/venv/lib/python3.12/site-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv(override=True)

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
documents = SimpleDirectoryReader("../data/paul_graham").load_data()
print("Document ID:", documents[0].doc_id)

Document ID: 59dc7cb8-54ac-4aa1-83ec-395986be013b


In [4]:
username = 'demo'
password = 'demo' 
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

In [5]:
vector_store = IRISVectorStore.from_params(
    connection_string=CONNECTION_STRING,
    table_name="paul_graham_essay",
    embed_dim=1536,  # openai embedding dimension
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# service_context = ServiceContext.from_defaults(
#     embed_model=embed_model, llm=None
# )

In [6]:
# This creates a persistent vector store (as a SQL table) under the hood and stores the document embeddings. This should only be called once!
index = VectorStoreIndex.from_documents(
    documents, 
    storage_context=storage_context, 
    show_progress=True, 
    # service_context=service_context,
)
query_engine = index.as_query_engine()

/Users/aryanput/dp/hackathon-2024/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 22/22 [00:01<00:00, 14.59it/s]


In [11]:
# If reconnecting to the vector store, use this: 

# index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# query_engine = index.as_query_engine()



In [18]:
# To add documents to an existing vector store:

# storage_context = StorageContext.from_defaults(vector_store=vector_store)
# for doc in documents:
#     index.insert(document=doc, storage_context=storage_context)

In [16]:
response = query_engine.query("What did the author do?")

In [17]:
import textwrap
print(textwrap.fill(str(response), 100))

The author wrote essays and also started to think about other things they could work on.


In [14]:
response = query_engine.query("What happened in the mid 1980s?")
print(textwrap.fill(str(response), 100))

AI was in the air in the mid 1980s.
